In [1]:
using PyPlot
using JuMP, Ipopt
using Combinatorics

In [2]:
include("../src/model.jl");
include("../src/diagnostics.jl");
include("../src/optimization.jl");
include("../src/plotting.jl");
include("../src/defaults.jl");

# Optimizating the Global Climate Action Policy Portfolio

In [3]:
model_og = ClimateModel();
max_deployment_og = Dict("mitigate"=>1., "remove"=>1., "geoeng"=>1., "adapt"=>1. /3.)

Dict{String,Float64} with 4 entries:
  "geoeng"   => 1.0
  "mitigate" => 1.0
  "remove"   => 1.0
  "adapt"    => 0.333333

In [4]:
controls = collect(keys(max_deployment_og));

In [5]:
control_permutations = combinations(controls) |> collect

15-element Array{Array{String,1},1}:
 ["geoeng"]                               
 ["mitigate"]                             
 ["remove"]                               
 ["adapt"]                                
 ["geoeng", "mitigate"]                   
 ["geoeng", "remove"]                     
 ["geoeng", "adapt"]                      
 ["mitigate", "remove"]                   
 ["mitigate", "adapt"]                    
 ["remove", "adapt"]                      
 ["geoeng", "mitigate", "remove"]         
 ["geoeng", "mitigate", "adapt"]          
 ["geoeng", "remove", "adapt"]            
 ["mitigate", "remove", "adapt"]          
 ["geoeng", "mitigate", "remove", "adapt"]

In [6]:
results = Dict()

for control_permutation in control_permutations    
    # Assert that some controls not be deployed
    max_deployment = copy(max_deployment_og)
    for control in controls
        if ~(control in control_permutation)
            max_deployment[control] = 0
        end
    end
    
    model = ClimateModel();
    @time m = optimize_controls!(model, max_deployment = max_deployment);
    
    results[control_permutation] = Dict("NPV" => discounted_total_control_cost(model), "status" => raw_status(m))
    
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Solve_Succeeded
 37.933928 seconds (111.14 M allocations: 6.318 GiB, 6.22% gc time)
Solve_Succeeded
  0.041463 seconds (89.13 k allocations: 3.963 MiB)
Infeasible_Problem_Detected
  0.748578 seconds (1.54 M allocations: 30.766 MiB, 4.47% gc time)
Infeasible_Problem_Detected
  0.815213 seconds (1.20 M allocations: 22.427 MiB)
Solve_Succeeded
  0.042789 seconds (92.70 k allocations: 4.069 MiB)
Solve_Succeeded
  0.030076 seconds (72.92 k allocations: 3.714 MiB)
Solve_Succeeded
  0.033105 seconds (89.84 k allocations: 4.007 MiB)
Solve_Succeeded
  0.047183 seconds (109.51 k allocations: 4.373 MiB)
Solve_Succee

In [7]:
min_value = Inf
for (index, value) in pairs(results)
    if (value["status"] == "Solve_Succeeded") & (value["NPV"] < min_value)
        min_value = value["NPV"]
    end
end

for (index, value) in pairs(results)
    if (value["status"] == "Solve_Succeeded")
        abs_value = Int64(round((value["NPV"])))
        rel_value = Int64(round((value["NPV"]/min_value - 1.)*100))
        print("$index, $abs_value, $rel_value%\n")
    else
        print("$index has no solution.\n")
    end
end


["geoeng", "mitigate", "remove"], 57, 4%
["remove"] has no solution.
["mitigate", "remove"], 63, 15%
["mitigate"], 87, 60%
["geoeng", "mitigate", "remove", "adapt"], 55, 0%
["geoeng", "adapt"], 294, 436%
["geoeng", "mitigate", "adapt"], 71, 29%
["mitigate", "remove", "adapt"], 60, 10%
["geoeng"], 411, 650%
["geoeng", "remove", "adapt"], 145, 165%
["mitigate", "adapt"], 81, 49%
["geoeng", "mitigate"], 75, 38%
["remove", "adapt"], 246, 350%
["geoeng", "remove"], 184, 236%
["adapt"] has no solution.
